# Words, words, words...

In [ ]:
%load_ext memory_profiler

In [1]:
# Imports
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing

import numpy as np
import pandas as pd

In [2]:
# Import custom code
import fld.io as flio
import fld.preds as flp

In [3]:
# Set data path
dat_path = "/Users/thomasdonoghue/Documents/UCSD/1-Classes/2016-2017/" \
           "2-Winter/CSE255_WebMining/Assignments/Assgn-2/Data/"

In [4]:
# Set labels
def create_label(row):
    if row['DEPARTURE_DELAY'] > 15: return 1
    else: return 0
    
def labels(df):
    return np.array(df['LABEL'])#.reshape(-1, 1)

In [28]:
def features_b(df):
    return np.array(flights_df[['DISTANCE','DAY_OF_WEEK']].values)
 
def features(df):
    
    h = np.array([1 if airline_hub[dat[1]] == dat[0] else 0 for dat in df[['ORIGIN_AIRPORT', 'AIRLINE']].values]).reshape(-1, 1)
    
    hn = np.array([airline_hubness[line] for line in df.AIRLINE.values]).reshape(-1, 1)  
    
    ce = np.array([airport_cen[port] for port in df.ORIGIN_AIRPORT.values]).reshape(-1, 1)
    cc = np.array([airport_clo_cen[port] for port in df.ORIGIN_AIRPORT.values]).reshape(-1, 1)
    de = np.array([airport_deg[port] for port in df.ORIGIN_AIRPORT.values]).reshape(-1, 1)
    nd = np.array([airport_ne_deg[port] for port in df.ORIGIN_AIRPORT.values]).reshape(-1, 1)
        
    oh_hour = flp.make_onehot_feat_mat(df, 'HOUR')
    
    oh_month = flp.make_onehot_feat_mat(df, 'MONTH')
    #oh_day = flp.make_onehot_feat_mat(df, 'DAY_OF_WEEK')
    
    oh_line = flp.make_onehot_feat_mat(df, 'AIRLINE')
    oh_o_port = flp.make_onehot_feat_mat(df, 'ORIGIN_AIRPORT')
    #oh_d_port = flp.make_onehot_feat_mat(df, 'DESTINATION_AIRPORT')
    
    lat_lon_dep = np.array(df[['LAT_DEP', 'LON_DEP']].values)
    #lat_lon_arr = np.array(df[['LAT_ARR', 'LON_ARR']].values)
    
    other = np.array(df[['DISTANCE']].values)

    #return np.hstack([oh_line, oh_o_port, oh_month, other])
    return np.hstack([oh_line, oh_o_port, oh_month, oh_hour, lat_lon_dep, other, h, hn, ce, cc, de, nd])

In [8]:
#%%memit
# Load data
airlines_df, airports_df, flights_df = flio.load_data(dat_path)#, N_flights=1000000)

In [9]:
# Update airlines & airports to use new updated ones
airlines_df = pd.read_json('new_airlines.json')
airports_df = pd.read_json('new_airports.json')

In [10]:
# Make dictionary look ups for airport and airline data to use as features. 

# Airlines data
airline_hub = airlines_df[['IATA_CODE', 'HUB']].set_index('IATA_CODE').to_dict()['HUB']
airline_hubness =  airlines_df[['IATA_CODE', 'HUBNESS']].set_index('IATA_CODE').to_dict()['HUBNESS']

# Airports data
airport_cen = airports_df[['IATA_CODE', 'CEN']].set_index('IATA_CODE').to_dict()['CEN']
airport_clo_cen = airports_df[['IATA_CODE', 'CLO_CEN']].set_index('IATA_CODE').to_dict()['CLO_CEN']
airport_deg = airports_df[['IATA_CODE', 'DEG']].set_index('IATA_CODE').to_dict()['DEG']
airport_ne_deg = airports_df[['IATA_CODE', 'NE_DEG']].set_index('IATA_CODE').to_dict()['NE_DEG']

#airline_hub = pd.Series(airlines_df.IATA_CODE.values, index=airlines_df.HUB).to_dict()
#airline_hub = dict((v,k) for k,v in airline_hub.items())
#airline_hub['UA'] = 'ORD'
#airline_hub['F9'] = 'DEN'
#airline_hubness = pd.Series(airlines_df.IATA_CODE.values, index=airlines_df.HUBNESS).to_dict()
#airline_hubness = dict((v,k) for k,v in airline_hubness.items())

# Airports data
#airport_cen = pd.Series(airports_df.IATA_CODE.values, index=airports_df.CEN)#.to_dict()
#airport_cen = dict((v,k) for k,v in airport_cen.items())
#airport_clo_cen = pd.Series(airports_df.IATA_CODE.values, index=airports_df.CLO_CEN).to_dict()
#airport_clo_cen = dict((v,k) for k,v in airport_clo_cen.items())
#airport_deg = pd.Series(airports_df.IATA_CODE.values, index=airports_df.DEG).to_dict()
#airport_deg = dict((v,k) for k,v in airport_deg.items())
#airport_ne_deg = pd.Series(airports_df.IATA_CODE.values, index=airports_df.NE_DEG).to_dict()
#airport_ne_deg = dict((v,k) for k,v in airport_ne_deg.items())

In [11]:
# Drop cancelled flights
flights_df = flio.drop_cancelled(flights_df)

#flights_df = flights_df[flights_df['CANCELLED'] != 1]
#flights_df = flights_df[np.isfinite(flights_df['DEPARTURE_DELAY'])]
#flights_df.dropna('DEPARTURE_DELAY')

In [12]:
# Filter data to keys of interest
keys = ['MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 'ORIGIN_AIRPORT',
       'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME',
       'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME',
       'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN',
       'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 'ARRIVAL_DELAY']
flights_df = flights_df[keys]

In [13]:
# Remove any flights with missing data
flights_df = flights_df.dropna()

In [14]:
# Select busy airports
flights_df = flio.select_busy_airports(flights_df)

In [15]:
# Add departure hour as a feature
flights_df['HOUR'] = np.floor(flights_df['SCHEDULED_DEPARTURE'].values/100).astype(int)

In [16]:
# Check which airports remain in database and select those in airport DataFrame
used_airports = set(flights_df.ORIGIN_AIRPORT.values)
airports_df = airports_df[airports_df.apply(lambda row: row['IATA_CODE'] in used_airports, axis=1)]

# Extract all aiports present in the data
airports = set(airports_df['IATA_CODE'])

In [17]:
def lat_lon(df):
    """Input: Airports DF."""
    
    airports = set(df['IATA_CODE'])
    
    lats = dict(); lons = dict();
    
    for p in airports:
        lats[p] = df.loc[df['IATA_CODE'] == p]['LATITUDE'].values[0]
        lons[p] = df.loc[df['IATA_CODE'] == p]['LONGITUDE'].values[0]

    return lats, lons

In [18]:
# Add Lat & Lon features to dataframe
lats, lons = lat_lon(airports_df)

In [19]:
# 'Center' of the USA, Lebanon, Kansas
cen_lat = 39.810213
cen_lon = -98.557317

#
#lats_cen = {key: lats[key] - cen_lat for key in lats.keys()}
#lons_cen = {key: lons[key] - cen_lon for key in lons.keys()}

# 
lats_cen = {key: abs(lats[key] - cen_lat) for key in lats.keys()}
lons_cen = {key: abs(lons[key] - cen_lon) for key in lons.keys()}

In [ ]:
# ?
lat_sc = preprocessing.MinMaxScaler((-1, 1))
lat_dat = np.array(list(lats_cen.values()))
lat_sc.fit(lat_dat)

lon_sc = preprocessing.MinMaxScaler((-1, 1))
lon_dat = np.array(list(lons_cen.values()))
lon_sc.fit(lon_dat)

In [ ]:
# ?
lats_cen_sc = {key: lat_sc.transform(np.array(lats_cen[key]).reshape(-1, 1))[0][0] for key in lats.keys()}
lons_cen_sc = {key: lon_sc.transform(np.array(lons_cen[key]).reshape(-1, 1))[0][0] for key in lons.keys()}

In [20]:
# Set departure airport lat & lon into flights df
flights_df['LAT_DEP'] = flights_df['ORIGIN_AIRPORT'].map(lats_cen)
flights_df['LON_DEP'] = flights_df['ORIGIN_AIRPORT'].map(lons_cen)

# Set arrival airport lat & lon into flights df
flights_df['LAT_ARR'] = flights_df['DESTINATION_AIRPORT'].map(lats_cen)
flights_df['LON_ARR'] = flights_df['DESTINATION_AIRPORT'].map(lons_cen)

In [21]:
# Split into train & test sets
n_flights = len(flights_df)
n_per_set = int(n_flights/2)
np.random.seed(0)
inds = np.random.permutation(flights_df.index)

# Split
flights_train_df = flights_df.loc[inds[:n_per_set]]
flights_test_df = flights_df.loc[inds[n_per_set:]]
del flights_df

In [31]:
# Create features representations
X_train = features(flights_train_df)
X_test = features(flights_test_df)

# Get labels for the train set
del_vals_train = flights_train_df['DEPARTURE_DELAY'].values
y_train = (del_vals_train > 15).astype(int)

# Get labels for the test set
del_vals_test = flights_test_df['DEPARTURE_DELAY'].values
y_test = (del_vals_test > 15).astype(int)

In [32]:
# Intialize and train the model
model = LogisticRegression(class_weight='balanced')
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [33]:
# Check model performance on train data
model.score(X_train, y_train)

0.59319121495891203

In [34]:
# Check model performance on test data
model.score(X_test, y_test)

0.59289706102013351

In [ ]:
# Check Confusion Matrix
preds = model.predict(X_test)
c_mat = confusion_matrix(y_test, preds)
print(c_mat)

In [ ]:
# TO BEAT: Check model performance on test data
model.score(X_test, y_test)